# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [79]:
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [47]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
datas = [data_0, data_1, data_2]

Посмотрим, как выгядят данные:

In [48]:
for data in datas:
    print(data.head(5))

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [49]:
for data in datas:
    print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

Видно, что у всех столбцов правильные типы данных

Посчитаем колличество дубликатов:

In [50]:
for data in datas:
    print(data.duplicated().sum())

0
0
0


Найдем количество пропусков:

In [51]:
for data in datas:
    print(data.isnull().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Поосмотрим, все ли `id` в таблицах уникальны: 

In [52]:
for data in datas:
    print(len(data['id'].unique()))

99990
99996
99996


In [53]:
for i in range(3):
    datas[i] = datas[i].drop_duplicates(subset=['id']).reset_index(drop=True)
    datas[i] = datas[i].drop(['id'], axis=1)

## Обучение и проверка модели

In [54]:
def make_pred(data):
    features = data.drop(['id', 'product'], axis = 1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=.25, random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train)
    return target_valid, pd.Series(model.predict(features_valid))

def mean_rmse (target, predictions):
    mse = mean_squared_error(target, predictions)
    print('Средний запас сырья:', predictions.mean())
    print('RMSE:', mse ** 0.5)

In [55]:
target_0, predict_0 = make_pred(data_0)
target_1, predict_1 = make_pred(data_1) 
target_2, predict_2 = make_pred(data_2)

In [57]:
print('Регион 0:')
mean_rmse(target_0, predict_0)
print('Регион 1:')
mean_rmse(target_1, predict_1)
print('Регион 2:')
mean_rmse(target_2, predict_2)

Регион 0:
Средний запас сырья: 92.59256778438035
RMSE: 37.5794217150813
Регион 1:
Средний запас сырья: 68.728546895446
RMSE: 0.893099286775617
Регион 2:
Средний запас сырья: 94.96504596800489
RMSE: 40.02970873393434


**Вывод: Самое низкое значение RMSE получилось у региона 1**

## Подготовка к расчёту прибыли

In [80]:
budget = 10**10
N_samples = 500
N_best_samples = 200
income_from_one = 450000
state = np.random.RandomState(12345)

In [59]:
budget_for_one = budget/N_best_samples
barrels = budget_for_one/income_from_one
print('Достаточный объём добычи для безубыточной разработки(тыс. баррелей):', barrels)

Достаточный объём добычи для безубыточной разработки(тыс. баррелей): 111.11111111111111


Получим средний истинный запас сырья в регионах:

In [60]:
targets = [target_0, target_1, target_2]
count = 0
for target in targets:
    print('Регион', count,':', target.mean())
    count += 1

Регион 0 : 92.07859674082927
Регион 1 : 68.72313602435997
Регион 2 : 94.88423280885438


In [61]:
def income(target, prediction):
    target = pd.Series(target).reset_index(drop=True)
    prediction = pd.Series(prediction).reset_index(drop=True)
    pred_sorted = prediction.sort_values(ascending=False)
    selected = target[pred_sorted.index][:N_best_samples]
    return ((income_from_one * selected.sum()) - budget)/10**9

In [62]:
zero_rev = income(target_0, predict_0)
print('Регион 0 (млрд):', zero_rev)

Регион 0 (млрд): 3.3208260431398506


In [63]:
first_rev = income(target_1, predict_1)
print('Регион 1 (млрд):', first_rev)

Регион 1 (млрд): 2.4150866966815108


In [64]:
second_rev = income(target_2, predict_2)
print('Регион 2 (млрд):', second_rev)

Регион 2 (млрд): 2.7103499635998327


**Вывод: Самая большая прибыль получилась у региона 0**

## Расчёт прибыли и рисков 

In [83]:
def bootstrap_method(target, predict):
    target = target.reset_index(drop=True)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        pred_subsample = predict[target_subsample.index]
        inc = income(target_subsample, pred_subsample)
        values.append(inc)
    q = 0.025
    values = pd.Series(values)
    lower = values.quantile(q)
    upper = values.quantile(1-q)
    interval = (lower, upper)
    mean = values.mean()

    print("Средняя выручка:", mean)
    print("2.5%-квантиль:", lower)
    print("Доверительный интервал:", interval)
    print("Риск убытка:", (values < 0).mean())

In [84]:
bootstrap_method(target_0, predict_0)

Средняя выручка: 0.3917836773766987
2.5%-квантиль: -0.14296964876776613
Доверительный интервал: (-0.14296964876776613, 0.8885694803482205)
Риск убытка: 0.07


In [85]:
bootstrap_method(target_1, predict_1)

Средняя выручка: 0.44709548279391526
2.5%-квантиль: 0.08157909628715802
Доверительный интервал: (0.08157909628715802, 0.8360008725599576)
Риск убытка: 0.012


In [86]:
bootstrap_method(target_2, predict_2)

Средняя выручка: 0.39532925955399834
2.5%-квантиль: -0.08941872554356879
Доверительный интервал: (-0.08941872554356879, 0.8872274522582164)
Риск убытка: 0.057


**Вывод: Наименьший риск убытка получился для Региона 1.**

## Общий вывод

**Получили следующие данные по регионам:** \
**RMSE:** 
- Регион 1 (0.893099286775617)
- Регион 0 (37.5794217150813)
- Регион 2 (40.02970873393434)

**Риск убытка:**
- Регион 1 (0.012)
- Регион 2 (0.057)
- Регион 0 (0.07)

**Из полученных данных можно сделать вывод, что лучшим регионом для разработки является Регион 1**